In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_excel('../data/Data-The-New-Gym-sorted.xlsx')
df

,Category,Club,DateKey,User number,Full name,Product Name,Product Type,Quantity,Regular price
0,Membership,The New Gym - Nguyen Chi Thanh,20210519,104002313,Nguyễn Phượng,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000
1,Membership,The New Gym - Ung Van Khiem,20210518,107000624,Phong Quân,Membership (6M_CN007_PO_2),Membership,1,1800000
2,Membership,The New Gym - Ung Van Khiem,20210518,107000625,Nguyễn Hậu,Membership (6M_CN007_PO_2),Membership,1,1800000
3,Membership,The New Gym - Ung Van Khiem,20210518,107000626,Nguyễn Ân,Membership (6M_CN007_PO_2),Membership,1,1800000
4,Membership,The New Gym - Nguyen Chi Thanh,20210517,105004342,Trần Ngân,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000
...,...,...,...,...,...,...,...,...,...
29994,Membership,The New Gym - Nguyen Chi Thanh,20200823,105000722,Trần Thư,Membership (1M_CN005_PO_2),Membership,1,319000
29995,Membership,The New Gym - Nguyen Chi Thanh,20200823,105000723,Tăng Tùng Hòa,Membership (1M_CN005_PO_2),Membership,1,319000
29996,Membership,The New Gym - Nguyen Chi Thanh,20200823,105000724,Trần Trâm,Membership (1M_CN005_PO_2),Membership,1,319000
29997,Membership,The New Gym - Nguyen Chi Thanh,20200823,105000726,Mai Tai,Membership (1M_CN005_PO_2),Membership,1,319000


In [3]:
filtered_data = df[~df['Product Name'].str.contains('Freeze')]
filtered_data.head()

,Category,Club,DateKey,User number,Full name,Product Name,Product Type,Quantity,Regular price
0,Membership,The New Gym - Nguyen Chi Thanh,20210519,104002313,Nguyễn Phượng,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000
1,Membership,The New Gym - Ung Van Khiem,20210518,107000624,Phong Quân,Membership (6M_CN007_PO_2),Membership,1,1800000
2,Membership,The New Gym - Ung Van Khiem,20210518,107000625,Nguyễn Hậu,Membership (6M_CN007_PO_2),Membership,1,1800000
3,Membership,The New Gym - Ung Van Khiem,20210518,107000626,Nguyễn Ân,Membership (6M_CN007_PO_2),Membership,1,1800000
4,Membership,The New Gym - Nguyen Chi Thanh,20210517,105004342,Trần Ngân,Membership (2M_CN005_BUY2GIVE1),Membership,1,798000


In [4]:
def process_membership(product_name):
    if 'Membership' in product_name or 'JoiningFee' in product_name:
        name, meta = product_name.split(' ')
        meta = meta.split('_')[0][1:] # remove (
        return f'{name}_{meta}'
    return product_name

filtered_data['Product Name'] = filtered_data['Product Name'].apply(process_membership)

C:\Users\nvatu\AppData\Local\Temp/ipykernel_8080/2636533179.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Product Name'] = filtered_data['Product Name'].apply(process_membership)


In [5]:
product_min_price = filtered_data[['Product Name', 'Regular price']].groupby('Product Name').min()

In [6]:
for row in product_min_price.iterrows():
    filtered_data.loc[filtered_data['Product Name'] == row[0], 'Regular price'] = row[1].values[0]

C:\Users\nvatu\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [7]:
filtered_data[
    ['DateKey', 'User number', 'Product Name', 'Quantity', 'Regular price']
].to_csv('../data/filtered_data.csv', index=False)

In [8]:
rfm = filtered_data.groupby('User number').sum().drop('DateKey', axis=1).reset_index()
rfm.columns = ['CustomerID', 'Frequency', 'Moneytary']
rfm['Frequency'] = filtered_data.groupby('User number').count().drop('DateKey', axis=1).reset_index()['Quantity']
rfm

,CustomerID,Frequency,Moneytary
0,101000281,6,14895000
1,101000282,4,1116000
2,101000283,5,1395000
3,101000320,1,558000
4,101000325,1,2800000
...,...,...,...
12359,107000620,1,1600000
12360,107000623,1,279000
12361,107000624,1,1600000
12362,107000625,1,1600000


In [9]:
data = filtered_data[['User number', 'DateKey']].set_index('DateKey').drop_duplicates(keep='last').reset_index()
data['DateKey'] = data['DateKey'].map(lambda x: f'{str(x)[:4]}-{str(x)[4:6]}-{str(x)[6:]}')

In [10]:
date_subtract = datetime(year=2022, month=3, day=31)

print(date_subtract)

2022-03-31 00:00:00


In [11]:
recency_unit = 7 # week

rfm['Recency'] = pd.to_datetime(data['DateKey'])
rfm['Recency'] = rfm['Recency'].map(lambda date: (date_subtract - date).days // recency_unit)

In [12]:
rfm

,CustomerID,Frequency,Moneytary,Recency
0,101000281,6,14895000,45
1,101000282,4,1116000,45
2,101000283,5,1395000,45
3,101000320,1,558000,45
4,101000325,1,2800000,45
...,...,...,...,...
12359,107000620,1,1600000,83
12360,107000623,1,279000,83
12361,107000624,1,1600000,83
12362,107000625,1,1600000,83


In [13]:
# thay đổi thứ tự của các cột
rfm = rfm[['CustomerID', 'Recency', 'Frequency', 'Moneytary']]
rfm

,CustomerID,Recency,Frequency,Moneytary
0,101000281,45,6,14895000
1,101000282,45,4,1116000
2,101000283,45,5,1395000
3,101000320,45,1,558000
4,101000325,45,1,2800000
...,...,...,...,...
12359,107000620,83,1,1600000
12360,107000623,83,1,279000
12361,107000624,83,1,1600000
12362,107000625,83,1,1600000


In [14]:
rfm.iloc[:, 1:]

,Recency,Frequency,Moneytary
0,45,6,14895000
1,45,4,1116000
2,45,5,1395000
3,45,1,558000
4,45,1,2800000
...,...,...,...
12359,83,1,1600000
12360,83,1,279000
12361,83,1,1600000
12362,83,1,1600000


In [15]:
# scale dữ liệu về khoảng [0, 1]
scaler = MinMaxScaler()
rfm.iloc[:, 1:] = scaler.fit_transform(rfm.iloc[:, 1:])

In [16]:
rfm

,CustomerID,Recency,Frequency,Moneytary
0,101000281,0.0,0.008787,0.198388
1,101000282,0.0,0.005272,0.014864
2,101000283,0.0,0.007030,0.018580
3,101000320,0.0,0.000000,0.007432
4,101000325,0.0,0.000000,0.037294
...,...,...,...,...
12359,107000620,1.0,0.000000,0.021311
12360,107000623,1.0,0.000000,0.003716
12361,107000624,1.0,0.000000,0.021311
12362,107000625,1.0,0.000000,0.021311


In [17]:
rfm.to_csv('../data/rfm_data.csv', index=False)